# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [16]:
boston_features.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,0.542096,1.0,296.0,15.3,1.000000,-1.275260
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,0.623954,2.0,242.0,17.8,1.000000,-0.263711
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,0.623954,2.0,242.0,17.8,0.989737,-1.627858
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,0.707895,3.0,222.0,18.7,0.994276,-2.153192
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,0.707895,3.0,222.0,18.7,1.000000,-1.162114


In [17]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns=['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20)

In [20]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [21]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train, y_train)
y_hat_test = linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [22]:
from sklearn.metrics import mean_squared_error
test_residuals = y_hat_test - y_test
test_residuals.head()

,target
332,3.354324
53,-1.316016
31,4.475552
449,4.317242
63,-1.978591


In [23]:
test_mse = mean_squared_error(y_test, y_hat_test)
test_mse

26.374654151771

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [25]:
def kfolds(data, k):
    # Force data as pandas dataframe           
    data = pd.DataFrame(data)
    num_obs = len(data)
    fold_size = num_obs//k
    leftovers = num_obs%k
    folds = []
    initial_obs = 0
    for fold_n in range(1, k+1):
        if fold_n <= leftovers:
        # Add 1 to account for leftovers
            fold = data.iloc[initial_obs: initial_obs + fold_size + 1]
            folds.append(fold)
            initial_obs += fold_size + 1
        else:
            fold = data.iloc[initial_obs: initial_obs + fold_size]
            folds.append(fold)
            initial_obs += fold_size
    return folds

### Apply it to the Boston Housing Data

In [26]:
# Make sure to concatenate the data again
bd = pd.concat([X.reset_index(drop=True), y], axis=1)

In [32]:
bf = kfolds(bd, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [33]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bf) if i != n])
    test = bf[n]
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print(train_errs)
print(test_errs)

[target    24.195577
dtype: float64, target    23.032087
dtype: float64, target    19.745073
dtype: float64, target    15.317101
dtype: float64, target    22.329973
dtype: float64]
[target    13.405145
dtype: float64, target    17.444017
dtype: float64, target    37.032711
dtype: float64, target    58.279544
dtype: float64, target    26.097989
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [34]:
from sklearn.model_selection import cross_val_score

crossfold5 = cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error')

In [35]:
crossfold5

array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [36]:
test_mse

26.374654151771

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!